# No feasible solution found.

In [1]:
# import sys
# sys.path
#  /Users/ryantalbot/opt/anaconda3/envs/tf2/bin/python3 -m pip install pyomo

In [46]:
import pandas as pd
import numpy as np
import pulp


In [47]:
# df = pd.read_csv('top_men.csv')

df = pd.read_csv('all_usa.csv')


In [51]:
df

,Name,Apparatus,Score
0,asher hong,VT,14.300
1,asher hong,VT,14.133
2,kameron nelson,VT,14.266
3,frederick richard,VT,14.450
4,kameron nelson,VT,13.633
...,...,...,...
1418,michael jaroh,FX,11.200
1419,frederick nathaniel richard,FX,14.166
1420,jeremy bischoff,FX,12.550
1421,frederick richard,FX,14.050


In [52]:
grouped = df.groupby(['Name', 'Apparatus'])['Score'].mean().reset_index()

In [54]:
df=grouped.copy()

In [56]:
df

,Name,Apparatus,Score
0,alex diab,FX,13.225000
1,alex diab,SR,14.158333
2,alex diab,VT,13.175000
3,alex karadzhov,FX,12.250000
4,alex karadzhov,HB,12.700000
...,...,...,...
441,zachary nunez,HB,13.150000
442,zachary nunez,PB,13.000000
443,zachary nunez,PH,13.133333
444,zachary nunez,SR,12.950000


In [64]:
df[df['Name']== 'yul moldauer']

,Name,Apparatus,Score
434,yul moldauer,FX,13.954125
435,yul moldauer,HB,12.458333
436,yul moldauer,PB,14.678571
437,yul moldauer,PH,13.607143
438,yul moldauer,SR,13.985714
439,yul moldauer,VT,14.250000


In [69]:
import pulp

# Create a PuLP problem
prob = pulp.LpProblem("GymnasticsAssignment", pulp.LpMaximize)

# Define the athletes and apparatus
athletes = df['Name'].unique()
known_apparatus = ['SR', 'PH', 'PB', 'HB', 'FX', 'VT']

# Create binary variables for each athlete and apparatus combination
athlete_vars = pulp.LpVariable.dicts("Athlete", (athletes, known_apparatus), 0, 1, pulp.LpBinary)

# Objective function: Maximize the total average score
athlete_avg_score = {athlete: df[df['Name'] == athlete]['Score'].values[0] for athlete in athletes}
prob += pulp.lpSum(athlete_avg_score[athlete] * pulp.lpSum(athlete_vars[athlete][apparatus] for apparatus in known_apparatus) for athlete in athletes)

# Constraint 1: Each athlete competes on up to 3 apparatus
for athlete in athletes:
    prob += pulp.lpSum(athlete_vars[athlete][apparatus] for apparatus in known_apparatus) <= 6  # Allow each athlete to compete on up to 3 apparatus

# Constraint 2: Each apparatus is used exactly three times
for apparatus in known_apparatus:
    prob += pulp.lpSum(athlete_vars[athlete][apparatus] for athlete in athletes) == 5

# Constraint 3: Select exactly 5 athletes
prob += pulp.lpSum(pulp.lpSum(athlete_vars[athlete][apparatus] for apparatus in known_apparatus) for athlete in athletes) == 5

# Solve the problem
prob.solve()

# Display the results
selected_athletes = [athlete for athlete in athletes if any(pulp.value(athlete_vars[athlete][apparatus]) == 1 for apparatus in known_apparatus)]

for apparatus in known_apparatus:
    print(f"Apparatus {apparatus}:")
    for athlete in selected_athletes:
        if any(pulp.value(athlete_vars[athlete][apparatus]) == 1 for apparatus in known_apparatus):
            scores = df[(df['Name'] == athlete) & (df['Apparatus'] == apparatus)]
            if not scores.empty:
                score = scores['Score'].values[0]
                print(f"{athlete} - {score}")
    print()

# Calculate the total average score
total_average_score = sum(athlete_avg_score[athlete] for athlete in selected_athletes)
print(f"Total Average Score: {total_average_score:.6f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/38776b52980c43e2ad378a07dc9a29ad-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/38776b52980c43e2ad378a07dc9a29ad-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 99 COLUMNS
At line 3232 RHS
At line 3327 BOUNDS
At line 3850 ENDATA
Problem MODEL has 94 rows, 522 columns and 1566 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Apparatus SR:
frederick richard - 14.0
paul juda - 13.95
vitaliy guimaraes - 13.6

Apparatus PH:
brandon dang - 14.1
frederick 

In [71]:
dfs=df.copy()

In [8]:
# import pulp
# import pandas as pd



# # Create a Linear Programming problem
# prob = pulp.LpProblem("Athlete_Selection", pulp.LpMaximize)

# # Define the decision variables (binary variables for each athlete on each apparatus)
# athletes = df['Name'].unique()
# apparatus = df['Apparatus'].unique()
# athlete_apparatus_vars = pulp.LpVariable.dicts("Select", [(athlete, app) for athlete in athletes for app in apparatus], 0, 1, pulp.LpInteger)

# # Define the objective function (maximize the sum of selected athletes)
# objective = pulp.lpSum(athlete_apparatus_vars[(athlete, app)] for athlete in athletes for app in apparatus)
# prob += objective

# # Constraint: Select exactly 5 athletes
# prob += pulp.lpSum(athlete_apparatus_vars[(athlete, app)] for athlete in athletes for app in apparatus) == 5

# # Constraint: Each athlete competes 3 times
# for athlete in athletes:
#     prob += pulp.lpSum(athlete_apparatus_vars[(athlete, app)] for app in apparatus) == 3

# # Constraint: Each apparatus is used 3 times
# for app in apparatus:
#     prob += pulp.lpSum(athlete_apparatus_vars[(athlete, app)] for athlete in athletes) == 3

# # Solve the optimization problem using the CBC solver
# prob.solve(pulp.PULP_CBC_CMD())

# # Extract the selected athletes and their respective apparatus
# selected_athletes = [athlete for athlete in athletes if any(pulp.value(athlete_apparatus_vars[(athlete, app)]) == 1 for app in apparatus)]

# # Print the selected athletes and their respective apparatus
# for athlete in selected_athletes:
#     selected_apparatus = [app for app in apparatus if pulp.value(athlete_apparatus_vars[(athlete, app)]) == 1]
#     print(f'{athlete} will compete on the following apparatus: {", ".join(selected_apparatus)}')


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/86911db8758b4ca3908ac382a3788965-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/86911db8758b4ca3908ac382a3788965-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 753 RHS
At line 781 BOUNDS
At line 902 ENDATA
Problem MODEL has 27 rows, 120 columns and 360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

donnell whittenburg will compete on the following apparatus: PB, FX, VT
alex diab will compete on the following apparatus: VT


In [75]:
# Apparatus SR:
# frederick richard - 14.0
# paul juda - 13.95
# yul moldauer - 13.985714

# Apparatus PH:
# brandon dang - 14.1
# stephen nedoroscik - 14.726142857142857
# vitaliy guimaraes - 14.35

# Apparatus PB:
# frederick richard - 14.25825
# vitaliy guimaraes - 13.9
# yul moldauer 14.678571

# Apparatus HB:
# frederick richard - 14.183333333333332
# paul juda - 13.649999999999999
# vitaliy guimaraes - 13.4

# Apparatus FX:
# frederick richard - 14.066666666666668
# paul juda - 14.65
# vitaliy guimaraes - 14.25

# Apparatus VT:
# paul juda - 14.325
# vitaliy guimaraes - 14.45
# yul moldauer - 14.250000

# fr=5
# pj=5
# vg=6
# bd=1
# sn=1



In [76]:
from pulp import LpMaximize, LpProblem, LpVariable

In [77]:
dfs

,Name,Apparatus,Score
0,alex diab,FX,13.225000
1,alex diab,SR,14.158333
2,alex diab,VT,13.175000
3,alex karadzhov,FX,12.250000
4,alex karadzhov,HB,12.700000
...,...,...,...
441,zachary nunez,HB,13.150000
442,zachary nunez,PB,13.000000
443,zachary nunez,PH,13.133333
444,zachary nunez,SR,12.950000


In [81]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

data=dfs.copy()


In [89]:
# Create a list of unique athlete names and assign unique IDs
unique_athletes = data["Name"].unique()
athlete_to_id = {athlete: i + 1 for i, athlete in enumerate(unique_athletes)}

# Create a list of unique apparatus
unique_apparatus = data["Apparatus"].unique()

# Create a matrix to store the performance data
num_athletes = len(unique_athletes)
num_apparatus = len(unique_apparatus)
performance_data = [[0] * num_apparatus for _ in range(num_athletes)]

# Populate the performance data matrix
for _, row in data.iterrows():
    athlete = row["Name"]
    apparatus = row["Apparatus"]
    athlete_id = athlete_to_id[athlete]
    apparatus_index = list(unique_apparatus).index(apparatus)
    if not pd.isna(row["Score"]):
        performance_data[athlete_id - 1][apparatus_index] = row["Score"]

# Define the ILP model
model = LpProblem(name="GymnasticsTeamSelection", sense=LpMaximize)

# Decision variables
x = {(i, j): LpVariable(name=f"x_{i}_{j}", cat="Binary") for i in range(1, num_athletes + 1) for j in range(1, num_apparatus + 1)}

# Objective function
model += sum(performance_data[i - 1][j - 1] * x[i, j] for i in range(1, num_athletes + 1) for j in range(1, num_apparatus + 1)), "Total_Value"

# Constraints
# Constraint 1: Select exactly 5 athletes
model += sum(x[i, j] for i in range(1, num_athletes + 1) for j in range(1, num_apparatus + 1)) == 5

# Constraint 2: Each apparatus is used exactly 3 times
for j in range(1, num_apparatus + 1):
    model += sum(x[i, j] for i in range(1, num_athletes + 1)) == 3

# Constraint 3: Athletes cannot compete on the same apparatus more than once
for i in range(1, num_athletes + 1):
    model += sum(x[i, j] for j in range(1, num_apparatus + 1)) <= 1

# Solve the ILP
model.solve()

# Check if a feasible solution exists
if model.status != 1:
    print("No feasible solution found.")
else:
    # Track the selected team
    selected_team = []

    for i in range(1, num_athletes + 1):
        for j in range(1, num_apparatus + 1):
            if x[i, j].varValue == 1:
                selected_team.append((unique_athletes[i - 1], unique_apparatus[j - 1]))

    # Print the selected team
    print("Selected Gymnastics Team:")
    for athlete, apparatus in selected_team:
        print(f"Athlete: {athlete}, Apparatus: {apparatus}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/bd3f04ad68d647f490baba5505c3ea76-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/bd3f04ad68d647f490baba5505c3ea76-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 99 COLUMNS
At line 3156 RHS
At line 3251 BOUNDS
At line 3774 ENDATA
Problem MODEL has 94 rows, 522 columns and 1566 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

No feasible solution found.
